# Assessing the Impact of Product Specifications and Brand Origin on the Pricing of Mechanical Keyboards in 2025

## Problem Statement
Global supply chains have undergone significant political and economic disruption in recent years, particularly in the technology and consumer electronics industries. 

Mechanical keyboards, an essential component of modern computing and creative work, have become a notable example of how Chinese manufacturers have entered the enthusiast market with competitive alternatives.

Historically, branding and Western design heritage contributed greatly to pricing. However, with increased transparency and direct-to-consumer models from Chinese factories, this may no longer hold true.

## Goal
This project aims to use mechanical keyboard listings as a case study to explore whether technical specifications and country/brand of origin still meaningfully influence pricing in 2025.

## Hypothesis
H₀ (Null Hypothesis): Product specifications and brand origin (e.g., Chinese vs Western brands) have no significant effect on price.

H₁ (Alternative Hypothesis): Product specifications and brand origin significantly affect price.

## Objectives
- Determine which features (e.g., switch type, brand, layout, connectivity) influence pricing.

- Analyze whether branding and origin remain significant predictors of pricing.

- Provide insights into broader trends of consumer electronics pricing post-supply-chain globalization.



# Seeing what data we are working with

In [1]:
import pandas as pd

df = pd.read_csv('../ds_capstone_project/keebfinder_keyboards_rev6.csv')
df_price_stats = pd.read_csv('../ds_capstone_project/brand_price_stats.csv')

df.head()

,brand_category,brand,title,price,layout,mount,hall_effect,hotswap,case_material,backlight,connectivity,screen,knob
0,unknown,0.01,0.01 Z62,$59,60%,Plate Mount,no,no,Metal (likely Aluminium),yes,Wired,no,no
1,unknown,0.01,0.01 Z62 Blank Blank,$59,60%,Plate Mount,no,no,Metal (likely Aluminium),yes,Wired,no,no
2,unknown,80retros,80retros GB65 X Click Inc,$169,65%,Gasket Mount,no,yes,Metal (likely Aluminium),no,Wired,no,no
3,unknown,80retros,80retros Pad Numpad X Click Inc,$129,Unknown,Gasket Mount,no,yes,Metal (likely Aluminium),no,Unknown,no,no
4,unknown,8BitDo,8BitDo Retro,$119,80%,Top Mount,no,yes,Metal (likely Aluminium),no,Wireless,no,yes


In [2]:
df.shape

(2368, 13)

In [3]:
df.isna().sum()

brand_category    0
brand             0
title             0
price             0
layout            0
mount             0
hall_effect       0
hotswap           0
case_material     0
backlight         0
connectivity      0
screen            0
knob              0
dtype: int64

## Preprocessing
- knowing there are missing values, determine what they are and deal with them

In [4]:
# check for all the missing values in the layout column
df['layout'].unique()

array(['60%', '65%', 'Unknown', '80%', '100%', '75%', '96%', '98%', '95%',
       '40%', '68%', '64%', '66%', '85%', '90%', '70%', '97%', '50%',
       '87%', '78%', '84%'], dtype=object)

In [5]:
#df['layout'] = df['layout'].str.replace(',', '')

In [6]:
#check for all the unique values in the mount column
df['mount'].unique()

array(['Plate Mount', 'Gasket Mount', 'Top Mount', 'Unknown',
       'Tray Mount', 'Sandwich Mount', 'Bottom Mount', 'PCB Mount'],
      dtype=object)

In [7]:
#df['mount'] = df['mount'].str.replace(',', '')

In [8]:
df['case_material'].unique()

array(['Metal (likely Aluminium)', 'Unspecified(likely ABS plastic)',
       'PCB Mount'], dtype=object)

In [9]:
#df['case_material'] = df['case_material'].str.replace(',', '')

In [10]:
# check for all the unique values in the 'connectivity' column
df['connectivity'].unique()

array(['Wired', 'Unknown', 'Wireless'], dtype=object)

In [11]:
#df['connectivity'] = df['connectivity'].str.replace(',', '')

In [12]:
# fill missing values with 'Unknown' for categorical columns
df['layout'].fillna('Unknown', inplace=True)
df['mount'].fillna('Unknown', inplace=True)
df['case_material'].fillna('Unknown', inplace=True)
df['connectivity'].fillna('Unknown', inplace=True)

/var/folders/04/wngkl13j3hj0s528zj5lpnj00000gn/T/ipykernel_7117/2699668920.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['layout'].fillna('Unknown', inplace=True)
/var/folders/04/wngkl13j3hj0s528zj5lpnj00000gn/T/ipykernel_7117/2699668920.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always beh

In [13]:
# minor mistake in the above code, lazy fix ^^
#df['case_material'].replace(to_replace="Unknown",
#         value="Unspecified(likely ABS plastic)", inplace=True)
#df['case_material'].replace(to_replace="Alu case",
#         value="Metal (likely Aluminium)", inplace=True)
#df.tail()


In [14]:
# # extract brand from title n make a new column
# df['brand'] = df['title'].str.split().str[0]
# # df.head()

# #reorder the columns to make brand go first
# df = df[['brand', 'title', 'price', 'layout', 'mount', 'hall_effect', 'hotswap', 'case_material', 'backlight', 'connectivity', 'screen', 'knob']]
# df.head()



## assigning brands a category

In [15]:
# categorize brands into pricing tiers (Provided by GPT-4)
brand_categories = {
    "premium": [
        "HHKB", "Realforce", "Geon", "Monokei", "Wooting", "Ergodox", "Keebwerk",
        "Protozoa", "Mechboards", "Meletrix", "MelGeek", "Dygma", "Keycult", "HIBI", "GMK"
    ],
    "midrange": [
        "Keychron", "Akko", "Varmilo", "Ducky", "Leopold", "IQUNIX", "Glorious", "Novelkeys",
        "Mistel", "Tex", "Vortex", "KBDFans", "KBParadise", "Nuphy", "Epomaker", "MelGeek"
    ],
    "budget": [
        "Ajazz", "Redragon", "Royalaxe", "Feker", "Skyloong", "Dareu", "Delux", "Zerodate",
        "Outemu", "Jamesdonkey", "Kemove", "AULA", "Langtu", "Womier", "Akko", "Dagk", "GamaKay",
        "MIIIW", "Darmoshark", "Monka", "Monsgeek", "Keydous", "Irok", "Newmen", "Niuniu"
    ]
}

# Create a mapping from brand name to category (Provided by GPT-4)
brand_to_category = {}
for category, brands in brand_categories.items():
    for brand in brands:
        brand_to_category[brand] = category

# Default uncategorized brands to 'unknown' (Provided by GPT-4)
unique_brands = df['brand'].dropna().unique()
for brand in unique_brands:
    if brand not in brand_to_category:
        brand_to_category[brand] = "unknown"

# Map the category to the dataframe
df['brand_category'] = df['brand'].map(brand_to_category)

In [16]:
# checking the unique values in the brand_category column
df['brand_category'].unique()

array(['unknown', 'budget', 'midrange', 'premium'], dtype=object)

In [17]:
df['brand'].unique()

array(['0.01', '80retros', '8BitDo', 'ACGAM', 'ATTACK', 'AULA', 'Ace',
       'Aesco', 'Aiglatson', 'Ajazz', 'Akko', 'Alpaca', 'Angel', 'Anne',
       'Apos', 'Azio', 'BAROCCOMISTEL', 'BASIC', 'Backspace', 'Bat',
       'Binepad', 'Black', 'CKW', 'Cerakey', 'Cherry', 'Chilkey', 'Cidoo',
       'ColorReco', 'Coolkiller', 'Corsair', 'Createkeebs', 'DOIO',
       'Dagk', 'Dareu', 'Daring', 'Darmoshark', 'Deadline', 'Dell',
       'Delux', 'Denshi', 'Design', 'Desktop', 'Double', 'DrunkDeer',
       'Dskeyboard', 'Ducky', 'Dukharo', 'Durgod', 'Dwarf', 'Dygma',
       'Endgame', 'Epomaker', 'Ergodox', 'Everglide', 'Evoworks', 'Fancy',
       'Fantech', 'Feker', 'Fiio', 'Filco', 'Firstblood', 'Fly',
       'Fl·esports', 'Fopato', 'Furycube', 'GDK', 'GMK', 'GMMK',
       'GamaKay', 'Ganss', 'Gateron', 'GeekElite', 'Geeksdontpanic',
       'Geon', 'Glorious', 'Gopolar', 'GravaStar', 'HEXCORE', 'HHKB',
       'HIBI', 'HMX', 'Haverworks', 'Heavy', 'Helix', 'Hexgears', 'Hiexa',
       'Homoo', 'H

In [18]:
# sahaja i want the brand_category to be the first column
#df = df[['brand_category','brand', 'title', 'price', 'layout', 'mount', 'hall_effect', 'hotswap', 'case_material', 'backlight', 'connectivity', 'screen', 'knob']]


In [19]:
# save hotfixes to a new dataframe over the CSV file
#df.to_csv('keebfinder_keyboards_rev5.csv', index=False)

## solving for brands that offer multiple tiers of products, eg high end, low end offerings


In [ ]:

# calculate min and max price per brand
#brand_price_stats = df.groupby('brand')['price'].agg(['min', 'max']).reset_index()
#brand_price_stats['price_range'] = brand_price_stats['min'].astype(str) + '–' + brand_price_stats['max'].astype(str)

# Save the brand_price_stats DataFrame to a CSV file
#brand_price_stats.to_csv('brand_price_stats.csv', index=False)

In [ ]:
df_price_stats.head()

,brand,min,max,price_range
0,0.01,$59,$59,$59–$59
1,80retros,$129,$169,$129–$169
2,8BitDo,$109,$69,$109–$69
3,ACGAM,$109,$99,$109–$99
4,ATTACK,$67,$89,$67–$89


In [ ]:
# finding the minimum prices range
min_min = df_price_stats['min'].min()
max_min = df_price_stats['min'].max()
min_min,max_min

('$10', '$99')